In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [ ]:
df = pd.read_csv("/content/NYC.csv")
df.head(8)

ParserError: Error tokenizing data. C error: Expected 11 fields in line 15249, saw 15


In [ ]:
df.shape

In [ ]:
df.info()

Observations:
1. missing values found in some columns
2. datetime columns are of object types

In [ ]:
null_per = df.isnull().sum()/df.shape[0]*100
null_per

In [ ]:
sns.heatmap(df.isnull(), cbar=False, cmap='viridis', yticklabels=False)

In [ ]:
def impute_nulls(col):
  df[col].fillna(df[col].mean(), inplace=True)

# Convert datetime columns to datetime objects with error handling
df['pickup_datetime'] = pd.to_datetime(df['pickup_datetime'])
df['dropoff_datetime'] = pd.to_datetime(df['dropoff_datetime'])

# Impute nulls
impute_nulls("dropoff_longitude")
impute_nulls("dropoff_latitude")

df['store_and_fwd_flag'].fillna(df['store_and_fwd_flag'].mode()[0], inplace=True)

impute_nulls("trip_duration")

df.isnull().sum()

In [ ]:
df.describe()

Observations:
1. minimum passenger count is 0 which is quite impossible
2. The maximum duration is approximately 3.5 million seconds in trip duration
3. Geospatial Columns have extremely wide ranges

In [ ]:
sum(df.duplicated())

In [ ]:
num_cols = df.select_dtypes(include=np.number).columns
outlier_summary = {}

for col in num_cols:
    Q1 = df[col].quantile(0.25)
    Q3 = df[col].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR

    outliers = ((df[col] < lower_bound) | (df[col] > upper_bound)).sum()
    outlier_summary[col] = {
        'lower_bound': lower_bound,
        'upper_bound': upper_bound,
        'outlier_count': outliers,
        'outlier_percentage': 100 * outliers / len(df)
    }

outlier_df = pd.DataFrame(outlier_summary).T
outlier_df

In [ ]:
df.vendor_id.value_counts()

In [ ]:
df.passenger_count.value_counts()

In [ ]:
invalid_passenger_counts = [0, 7, 8, 9]

df = df[~df['passenger_count'].isin(invalid_passenger_counts)]

print(df['passenger_count'].value_counts())
print(df.shape)

In [ ]:
lat_min, lat_max = 40.49, 40.92
lon_min, lon_max = -74.27, -73.68

pickup_outliers = df[
    (df['pickup_latitude'] < lat_min) | (df['pickup_latitude'] > lat_max) |
    (df['pickup_longitude'] < lon_min) | (df['pickup_longitude'] > lon_max)
]

print("Pickup outliers:", pickup_outliers.shape)
pickup_outliers.head()


In [ ]:
dropoff_outliers = df[
    (df['dropoff_latitude'] < lat_min) | (df['dropoff_latitude'] > lat_max) |
    (df['dropoff_longitude'] < lon_min) | (df['dropoff_longitude'] > lon_max)
]

print("Dropoff outliers:", dropoff_outliers.shape)
dropoff_outliers.head()

In [ ]:
# Haversine function (km) - returns a NumPy array
def haversine(lon1, lat1, lon2, lat2):
    R = 6371  # Earth radius in km
    lon1, lat1, lon2, lat2 = map(np.radians, [lon1, lat1, lon2, lat2])
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = np.sin(dlat / 2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon / 2)**2
    return R * 2 * np.arcsin(np.sqrt(a))

trip_distances = haversine(
    df['pickup_longitude'], df['pickup_latitude'],
    df['dropoff_longitude'], df['dropoff_latitude']
)

coord_outlier = (
    (~df['pickup_latitude'].between(lat_min, lat_max) |
     ~df['pickup_longitude'].between(lon_min, lon_max) |
     ~df['dropoff_latitude'].between(lat_min, lat_max) |
     ~df['dropoff_longitude'].between(lon_min, lon_max))
)

df = df[~(coord_outlier & (trip_distances > 150))]

Q1 = df['trip_duration'].quantile(0.25)
Q3 = df['trip_duration'].quantile(0.75)
IQR = Q3 - Q1
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

df = df[(df['trip_duration'] >= lower_bound) & (df['trip_duration'] <= upper_bound)]

print(df.shape)

In [ ]:
df.store_and_fwd_flag.value_counts()

In [ ]:
plt.figure(figsize=(5,5))
df['store_and_fwd_flag'].value_counts().plot(
    kind='pie',
    autopct='%1.1f%%',
    startangle=90,
    colors=['skyblue', 'salmon'],
    labels=['N', 'Y']
)
plt.ylabel('')
plt.title('Store and Forward Flag Distribution')

In [ ]:
df['store_and_fwd_flag'] = df['store_and_fwd_flag'].map({'N': 0, 'Y': 1})
df.tail()

In [ ]:
print(df['pickup_datetime'].dtype)

# Check date range
print(df['pickup_datetime'].min(), df['pickup_datetime'].max())

# Check for negative trip durations
print((df['dropoff_datetime'] < df['pickup_datetime']).sum())

# Check duplicates
print(df.duplicated(subset=['pickup_datetime', 'dropoff_datetime']).sum())

In [ ]:
df = df.drop_duplicates(subset=['pickup_datetime', 'dropoff_datetime'])

In [ ]:
plt.figure(figsize=(12,6))
df.set_index('pickup_datetime').resample('D').size().plot()
plt.title('Daily Trip Counts')
plt.xlabel('Date')
plt.ylabel('Number of Trips')

In [ ]:
# Daily trip counts
daily_counts = df.set_index('pickup_datetime').resample('D').size()

# Find lowest day
min_day = daily_counts.idxmin()
print("Lowest trip day:", min_day, "with", daily_counts.min(), "trips")

# Look at hourly distribution for that day
hourly_counts = df[df['pickup_datetime'].dt.date == min_day.date()] \
    .set_index('pickup_datetime') \
    .resample('H').size()
print(hourly_counts)

January 23, 2016 was the day of the big NYC blizzard (“Winter Storm Jonas”).
Almost all public transport and taxi service shut down in the afternoon, so the trip counts crashed to nearly zero after 3 PM.

That’s why you see:

Normal trip volume until about 2 PM,

Then a steep drop,

Zero trips in evening/night hours.

In [ ]:
mask = df['pickup_datetime'].dt.normalize() != pd.to_datetime('2016-01-23')
df = df[mask]
mask_dropoff = df['dropoff_datetime'].dt.normalize() != pd.to_datetime('2016-01-23')
df = df[mask_dropoff]

In [ ]:
df[df['pickup_datetime'].dt.normalize() == pd.to_datetime('2016-01-23')]

In [ ]:
df[df['dropoff_datetime'].dt.normalize() == pd.to_datetime('2016-01-23')]

In [ ]:
df.head()

In [ ]:
# Pickup features
df['pickup_year'] = df['pickup_datetime'].dt.year
df['pickup_month'] = df['pickup_datetime'].dt.month
df['pickup_day'] = df['pickup_datetime'].dt.day
df['pickup_dayofweek'] = df['pickup_datetime'].dt.dayofweek
df['pickup_hour'] = df['pickup_datetime'].dt.hour
df['pickup_weekofyear'] = df['pickup_datetime'].dt.isocalendar().week.astype(int)
df['is_weekend'] = (df['pickup_datetime'].dt.dayofweek >= 5).astype(int)

# Dropoff features
df['dropoff_year'] = df['dropoff_datetime'].dt.year
df['dropoff_month'] = df['dropoff_datetime'].dt.month
df['dropoff_day'] = df['dropoff_datetime'].dt.day
df['dropoff_dayofweek'] = df['dropoff_datetime'].dt.dayofweek
df['dropoff_hour'] = df['dropoff_datetime'].dt.hour
df['dropoff_weekofyear'] = df['dropoff_datetime'].dt.isocalendar().week.astype(int)
df['dropoff_is_weekend'] = (df['dropoff_datetime'].dt.dayofweek >= 5).astype(int)

df = df.drop(['pickup_datetime', 'dropoff_datetime'], axis=1)
df.head()

In [ ]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler

df_model = df.drop(columns=['id']).copy()

# Log-transform trip_duration to reduce skew
df_model['trip_duration'] = np.log1p(df_model['trip_duration'])

# Scale continuous features (z-score standardization)
scaler = StandardScaler()
df_scaled = pd.DataFrame(
    scaler.fit_transform(df_model),
    columns=df_model.columns,
    index=df_model.index
)

# Normalize to [0, 1]
normalizer = MinMaxScaler()
df_normalized = pd.DataFrame(
    normalizer.fit_transform(df_scaled),
    columns=df_scaled.columns,
    index=df_scaled.index
)

df = df_normalized

print("Final df shape:", df.shape)
print(df.head())

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score


X = df.drop(columns=['trip_duration'])  # drop target + id
y = df['trip_duration']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

def evaluate_model(model, name):
    model.fit(X_train, y_train)
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)

    results = {
        'Model': name,
        'Train RMSE': np.sqrt(mean_squared_error(y_train, y_train_pred)),
        'Test RMSE': np.sqrt(mean_squared_error(y_test, y_test_pred)),
        'Train R²': r2_score(y_train, y_train_pred),
        'Test R²': r2_score(y_test, y_test_pred)
    }
    return results

# Example usage
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline

results = []
results.append(evaluate_model(LinearRegression(), "Linear Regression"))
results.append(evaluate_model(make_pipeline(PolynomialFeatures(degree=2), LinearRegression()), "Polynomial Regression (deg=2)"))
results.append(evaluate_model(Ridge(alpha=1.0), "Ridge Regression"))

pd.DataFrame(results)


In [ ]:
from sklearn.linear_model import Ridge, Lasso
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV

# Create a pipeline for Ridge with Polynomial Features
ridge_poly_pipeline = Pipeline([
    ('poly', PolynomialFeatures(degree=2)),
    ('ridge', Ridge())
])
param_grid_ridge = {'ridge__alpha': [0.1, 1.0, 10.0, 100.0]}
ridge_cv = GridSearchCV(ridge_poly_pipeline, param_grid_ridge, cv=5, scoring='r2', n_jobs=-1)
ridge_cv.fit(X_train, y_train)

print("Best Ridge Polynomial parameters:", ridge_cv.best_params_)
print("Optimized Ridge Polynomial Test R² score:", ridge_cv.best_estimator_.score(X_test, y_test))
print("-" * 50)

In [ ]:
# Create a pipeline for Lasso with Polynomial Features
lasso_poly_pipeline = Pipeline([
    ('poly', PolynomialFeatures(degree=2)),
    ('lasso', Lasso())
])
param_grid_lasso = {'lasso__alpha': [0.001, 0.01, 0.1, 1.0]}
lasso_cv = GridSearchCV(lasso_poly_pipeline, param_grid_lasso, cv=5, scoring='r2', n_jobs=-1)
lasso_cv.fit(X_train, y_train)

print("Best Lasso Polynomial parameters:", lasso_cv.best_params_)
print("Optimized Lasso Polynomial Test R² score:", lasso_cv.best_estimator_.score(X_test, y_test))

In [ ]:
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
import xgboost as xgb

results = []
results.append(evaluate_model(RandomForestRegressor(n_estimators=100, random_state=42), "Random Forest Regressor"))
results.append(evaluate_model(GradientBoostingRegressor(n_estimators=100, learning_rate=0.1, max_depth=3, random_state=42), "Gradient Boosting Regressor"))
results.append(evaluate_model(xgb.XGBRegressor(n_estimators=100, learning_rate=0.1, max_depth=3, random_state=42), "XGBoost Regressor"))

new_results_df = pd.DataFrame(results)
print(new_results_df)

In [ ]:
from sklearn.model_selection import RandomizedSearchCV

# Random Forest
rf = RandomForestRegressor(random_state=42)
param_dist_rf = {
    "n_estimators": [100, 300, 500],
    "max_depth": [None, 10, 20, 30],
    "min_samples_split": [2, 5, 10],
    "min_samples_leaf": [1, 2, 4],
    "max_features": ["auto", "sqrt", 0.5]
}
rf_cv = RandomizedSearchCV(rf, param_dist_rf, cv=3, n_iter=20,
                           scoring="r2", n_jobs=-1, random_state=42)
rf_cv.fit(X_train, y_train)
print("Best RF params:", rf_cv.best_params_)
print("RF Test R²:", rf_cv.best_estimator_.score(X_test, y_test))

# XGBoost
xgb_reg = xgb.XGBRegressor(random_state=42, objective="reg:squarederror")
param_dist_xgb = {
    "n_estimators": [100, 300, 500],
    "max_depth": [3, 5, 7],
    "learning_rate": [0.01, 0.05, 0.1],
    "subsample": [0.6, 0.8, 1.0],
    "colsample_bytree": [0.6, 0.8, 1.0],
    "reg_lambda": [0.1, 1, 10]
}
xgb_cv = RandomizedSearchCV(xgb_reg, param_dist_xgb, cv=3, n_iter=20,
                            scoring="r2", n_jobs=-1, random_state=42)
xgb_cv.fit(X_train, y_train)
print("Best XGB params:", xgb_cv.best_params_)
print("XGB Test R²:", xgb_cv.best_estimator_.score(X_test, y_test))
